# Requirement Specification for Product Allocation Algorithm

## Objective

1. To minimize the maximum transportation time across all warehouse-order allocations as represented in the Allocation Matrix.

## Input Parameters

1. **Transport Time Matrix (X)**: A matrix that represents the transport time (per unit) between each warehouse and each order.
2. **Demand Array (Y)**: An array that specifies the demand for each order.
3. **Stock Array (Z)**: An array that details the current stock of the product in each warehouse.

## Output Parameters

1. **Allocation Matrix (A)**: A matrix indicating the volume of the product that will be transported from each warehouse to each order.
2. **Minimized Maximum Transport Time**: The smallest maximum transport time that can be achieved. where transport time is calculated by multiplying the transport time per unit with the volume of the product transported.
## Constraints



1. For each order $i$, $\sum_{j} A_{ij} = Y_i$, where $A_{ij}$ is the volume allocated from warehouse $j$ to order $i$.
2. For each warehouse $j$, $\sum_{i} A_{ij} \leq Z_j$, where $Z_j$ is the inventory in warehouse $j$.
3. For each warehouse $j$ and order $i$, $A_{ij} \geq 0$.


## Algorithm Documentation: Optimal Product Allocation from Multiple Warehouses to Multiple Orders

---

#### Overview:

The algorithm tackles the problem of optimally allocating a specific type of product from various warehouses to multiple orders. The objective is to minimize the maximum transportation time using Integer Linear Programming (ILP).

---
#### Mathematical Formulation:

**Variables**:

- $ X $: Transport time matrix of size $ n \times m $, where $ X_{i,j} $ is the time taken to transport the product from warehouse $ j $ to order $ i $.
  
- $ Y $: Demand vector of size $ n $, where $ Y_i $ is the demand for order $ i $.

- $ Z $: Inventory vector of size $ m $, where $ Z_j $ denotes the available stock in warehouse $ j $.

- $ A $: Allocation matrix of size $ n \times m $, where $ A_{i,j} $ is the volume of product allocated from warehouse $ j $ to order $ i $. This is our primary decision variable.

- $ M $: A scalar variable representing the maximum transportation time among all allocations. 

---

**Objective Function**:

$$ \min M $$
   
The primary objective is to minimize $ M $, which captures the maximum transportation time in the entire allocation matrix.

---

**Constraints**:

1. **Max Transport Time Constraint**:
   
$$ M \geq X_{i,j} \times A_{i,j} \quad \forall i \in \{1,2,\ldots,n\}, j \in \{1,2,\ldots,m\} $$
    
For every combination of order $ i $ and warehouse $ j $, this constraint ensures that $ M $ is always greater than or equal to the product of the transport time and the allocated volume.

2. **Order Demand Constraint**:

$$ \sum_{j=1}^{m} A_{i,j} = Y_i \quad \forall i \in \{1,2,\ldots,n\} $$

This ensures that the total allocation for each order matches its demand.

3. **Warehouse Inventory Constraint**:

$$ \sum_{i=1}^{n} A_{i,j} \leq Z_j \quad \forall j \in \{1,2,\ldots,m\} $$

This guarantees that allocations from each warehouse do not exceed its inventory.

---

#### Algorithm Description:

### Step 1: Input Validation
- Verify the inputs (Transport Time Matrix $X$, Demand Array $Y$, Stock Array $Z$) to ensure they are valid and consistent with each other in terms of dimensions and values.

### Step 2: Initialize Allocation Matrix (A)
- Create an empty matrix $A$ with the same dimensions as the Transport Time Matrix $X$.

### Step 3: Calculate Initial Allocation
- Distribute products according to an initial algorithm, perhaps starting with the shortest transport times first, while adhering to the constraints.
  
### Step 4: Optimize for Minimum Maximum Transport Time
- Use an optimization algorithm (such as Linear Programming or other optimization techniques) to adjust the initial allocation in $A$. The objective is to minimize the maximum transportation time, which is calculated by the Hadamard product of $A$ and $X$.

### Step 5: Validate Final Allocation
- Ensure that the final allocation $A$ satisfies all constraints:
  1. Sum of allocations for each order $i$ should be equal to $Y_i$.
  2. Sum of allocations for each warehouse $j$ should not exceed $Z_j$.
  3. All elements in $A$ should be greater than or equal to 0.

### Step 6: Calculate Minimized Maximum Transport Time
- Calculate the minimized maximum transport time as per the final $A$ and $X$.

### Step 7: Output
- Return the Allocation Matrix $A$ and the minimized maximum transport time as the outputs.

After laying out these steps, the next phases would involve implementing the code, followed by testing with sample data to validate that the algorithm is functioning as expected.

---

#### Conclusion: 

**TODO**: need update, on how outer system call this API, specifically, it call this algo for diff product.

By using this algorithm, supply chain managers can make informed decisions about product allocation, optimizing transportation time, and ensuring timely deliveries. The mathematical formulation provides clarity on the constraints and objectives, making the approach more transparent and understandable.

## implementation

### Step 1: Input Validation
- Verify the inputs (Transport Time Matrix \(X\), Demand Array \(Y\), Stock Array \(Z\)) to ensure they are valid and consistent with each other in terms of dimensions and values. with these test data:

In [1]:
import numpy as np

def validate_inputs(X, Y, Z):
    """
    Validates the input parameters for the product allocation problem.
    
    :param X: Transport Time Matrix (2D NumPy array)
    :param Y: Demand Array (1D NumPy array)
    :param Z: Stock Array (1D NumPy array)
    :return: A boolean value indicating the validity of the inputs and a message string.
    """
    # Check if X is a 2D array
    if X.ndim != 2:
        return False, "X must be a 2D array currently"
    
    # Check if Y and Z are 1D arrays
    if Y.ndim != 1 or Z.ndim != 1:
        return False, "Y and Z must be 1D arrays currently"
    
    # Check if the dimensions of X, Y, and Z are compatible
    rows_X, cols_X = X.shape
    if len(Y) != rows_X or len(Z) != cols_X:
        return False, "Dimensions of X, Y, and Z are not compatible"
    
    # Check if all elements are non-negative
    if np.any(X < 0) or np.any(Y < 0) or np.any(Z < 0):
        return False, "All elements in X, Y, and Z must be non-negative"
    
    return True, "Inputs are valid"

# Test data definitions
test_data = [
    {
        'X': np.array([[4, 2], [3, 2]]),
        'Y': np.array([1, 1]),
        'Z': np.array([3, 4]),
        'priority_order': np.array([0, 1]),
        'description': "All orders can be satisfied, no priority needed"
    },
    {
        'X': np.array([[2, 8], [1, 4]]),
        'Y': np.array([10, 5]),
        'Z': np.array([5, 5]),
        'priority_order': np.array([0, 1]),
        'description': "Not all orders can be satisfied, and priority comes into play"
    },
    {
        'X': np.array([[1, 1], [1, 2]]),
        'Y': np.array([1, 1]),
        'Z': np.array([1, 1]),
        'priority_order': np.array([0, 1]),
        'description': "All orders can be satisfied, but not in a greedy manner"
    },
    {
        'X': np.array([[1], [0], [2]]),
        'Y': np.array([1, 1, 2]),
        'Z': np.array([2]),
        'priority_order': np.array([2, 1, 0]),
        'description': "Not all orders can be satisfied, and priority comes into play"
    },
    {
        'X': np.array([[1]]),
        'Y': np.array([1]),
        'Z': np.array([1]),
        'priority_order': np.array([0]),
        'description': "All orders can be satisfied, no priority needed"
    },
    {
        'X': np.array([[1, 1], [0, 0]]),
        'Y': np.array([1, 1]),
        'Z': np.array([1, 1]),
        'priority_order': np.array([1, 0]),
        'description': "Not all orders can be satisfied, but all warehouses can contribute to the high priority order"
    },
    {
        'X': np.array([[1., 4.],
                    [2., 2.],
                    [4., 2.],
                    [2., 1.],
                    [1., 4.]]), 
        'Y': np.array([46., 26.,  6., 27., 17.]),
        'Z': np.array([2., 18.]),
        'priority_order': np.array([3, 4, 1, 0, 2]),
        'description': "Complex case"
    }
]

# Validate the test data
for i, case in enumerate(test_data):
    is_valid, message = validate_inputs(case['X'], case['Y'], case['Z'])
    print(f"Test case {i+1} ({case['description']}): {message}")


Test case 1 (All orders can be satisfied, no priority needed): Inputs are valid
Test case 2 (Not all orders can be satisfied, and priority comes into play): Inputs are valid
Test case 3 (All orders can be satisfied, but not in a greedy manner): Inputs are valid
Test case 4 (Not all orders can be satisfied, and priority comes into play): Inputs are valid
Test case 5 (All orders can be satisfied, no priority needed): Inputs are valid
Test case 6 (Not all orders can be satisfied, but all warehouses can contribute to the high priority order): Inputs are valid
Test case 7 (Complex case): Inputs are valid



### Step 2: Initialize Allocation Matrix (A)
- Create an empty matrix \(A\) with the same dimensions as the Transport Time Matrix \(X\).

### Step 3: Calculate Initial Allocation
- Distribute products according to an initial algorithm, perhaps starting with the shortest transport times first, while adhering to the constraints.

In [2]:
import numpy as np



def initial_allocation(data):
    """
    Perform initial allocation based on the minimum transport times.

    :param data: A dictionary containing X, Y, and Z
    :return: Initial Allocation Matrix A, Remaining Stock, Remaining Demand
    """
    X = data['X']
    Y = data['Y']
    Z = data['Z']

    A = np.zeros_like(X)
    remaining_Y = np.copy(Y)
    remaining_Z = np.copy(Z)

    sorted_idx = np.unravel_index(np.argsort(X, axis=None), X.shape)
    for i, j in zip(*sorted_idx):
        if remaining_Y[i] == 0 or remaining_Z[j] == 0:
            continue
        allocate_amount = min(remaining_Y[i], remaining_Z[j])
        A[i, j] += allocate_amount
        remaining_Y[i] -= allocate_amount
        remaining_Z[j] -= allocate_amount

    return A, remaining_Z, remaining_Y



### Step 4: Optimize for Minimum Maximum Transport Time
- Use an optimization algorithm (such as Linear Programming, bi-paritite graph matching, networkflow or other non-heuristic optimization techniques) to adjust the initial allocation in \(A\). The objective is to minimize the maximum transportation time, which is calculated by the Hadamard product of \(A\) and \(X\).
- Note that greedy is not capable in some case like:

In [3]:
import pulp
import numpy as np

def optimize_allocation(data):
    X = data['X']
    Y = data['Y']
    Z = data['Z']
    priority_order = data['priority_order']
    
    num_orders, num_warehouses = X.shape

    # Initialize the ILP problem
    prob = pulp.LpProblem("Product_Allocation", pulp.LpMinimize)

    # Decision Variables (integer type specified)
    A_vars = pulp.LpVariable.dicts("Allocation", [(i, j) for i in range(num_orders) for j in range(num_warehouses)],
                                   0, cat='Integer')  # non-negative integers
    M = pulp.LpVariable("Max_Transport_Time", 0)  # non-negative

    # Objective Function
    prob += M, "Objective is to Minimize the Maximum Transport Time"

    # Constraints
    # Set M to be greater than every element in the Hadamard product of X and A
    for i in range(num_orders):
        for j in range(num_warehouses):
            prob += M >= X[i][j] * A_vars[(i, j)]

    # Adjusted Constraint: Ensure sum of allocations for each order is exactly the demand
    for i in range(num_orders):
        prob += pulp.lpSum([A_vars[(i, j)] for j in range(num_warehouses)]) == Y[i]
        
    # Ensure sum of allocations for each warehouse is <= inventory
    for j in range(num_warehouses):
        prob += pulp.lpSum([A_vars[(i, j)] for i in range(num_orders)]) <= Z[j]

    # Solve
    prob.solve()


    # Extract values of A_vars into a matrix form
    A = np.zeros((num_orders, num_warehouses))
    for i, j in A_vars:
        A[i, j] = A_vars[(i, j)].varValue

    return A



- add consideration for priority: :
    1. ignore priority when all order can be satisfied, find the global optimal solution in this case.
    2. allocate entirely according toorder priority when some of order are unstisfiable, (ignore optimal solution, provide as many as possible inventory for higher priority order)

In [4]:
def allocate_based_on_priority(Y, Z, priority_order):
    A = np.zeros((len(Y), len(Z)))
    for order_index in priority_order:
        for warehouse_index in range(len(Z)):
            allocation = min(Z[warehouse_index], Y[order_index])
            A[order_index][warehouse_index] = allocation
            Z[warehouse_index] -= allocation
            Y[order_index] -= allocation
    return A

def optimize_allocation_priority(data):
    X = data['X']
    Y = data['Y'].copy()  # We make a copy since we modify it
    Z = data['Z'].copy()  # We make a copy since we modify it
    priority_order = data['priority_order']
    
    A, remaining_Z, remaining_Y = initial_allocation(data)
    
    if np.sum(remaining_Y) > 0:  # Not all orders can be satisfied
        A = allocate_based_on_priority(Y, Z, priority_order)
    else:  # All orders can be satisfied
        A = optimize_allocation(data)
    
    return A


### Step 5: Validate Final Allocation
- Ensure that the final allocation \(A\) satisfies all constraints:
  1. Sum of allocations for each order \(i\) should be equal to \(Y_i\).
  2. Sum of allocations for each warehouse \(j\) should not exceed \(Z_j\).
  3. All elements in \(A\) should be greater than or equal to 0.

In [5]:
def validate_allocation_priority(A, Y, Z, priority_order, expected_A=None):
    # Validate if sum of allocations for each order i is equal to Yi
    for i, y in enumerate(Y):
        if np.sum(A[:, i]) != y:
            if np.sum(A) == np.sum(Z):  # All inventory is used
                return False, f"Order {i} allocation mismatch considering priority. Expected: {y}, Got: {np.sum(A[:, i])}. But all inventory was used."
            else:
                return False, f"Order {i} allocation mismatch considering priority. Expected: {y}, Got: {np.sum(A[:, i])}."
    
    # Validate if sum of allocations from each warehouse j does not exceed Zj
    for j, z in enumerate(Z):
        if np.sum(A[j, :]) > z:
            return False, f"Warehouse {j} over allocation. Expected: <= {z}, Got: {np.sum(A[j, :])}."
    
    # Validate if all elements in A are greater than or equal to 0
    if np.any(A < 0):
        return False, "Negative allocation detected."
    
    # If expected_A is provided, compare with the given A
    if expected_A is not None:
        if not np.array_equal(A, expected_A):
            return False, "The given allocation matrix does not match the expected result."

    return True, "All constraints satisfied."




### Step 6: Calculate Minimized Maximum Transport Time
- Calculate the minimized maximum transport time as per the final \(A\) and \(X\).

### Step 7: Output
- Return the Allocation Matrix \(A\) and the minimized maximum transport time as the outputs.

In [6]:
import numpy as np

def compute_minimized_max_transport_time(A, X):
    """
    Compute the minimized maximum transport time based on allocation matrix A and transport time matrix X.
    
    Args:
    - A (numpy.ndarray): Allocation matrix.
    - X (numpy.ndarray): Transport time matrix.
    
    Returns:
    - float: Minimized maximum transport time.
    """
    # Compute the Hadamard product
    hadamard_product = np.multiply(A, X)
    
    # Find and return the maximum value in the matrix
    return np.max(hadamard_product)

def main_allocation_function(case):
    """
    Optimize the product allocation and compute minimized maximum transport time.
    
    Args:
    - case (dict): Dictionary containing priority order, transport time matrix X, demand array Y, and stock array Z.
    
    Returns:
    - dict: Dictionary containing optimized allocation matrix A and minimized maximum transport time.
    """
    
    # Optimize allocation considering priority
    A_optimized = optimize_allocation_priority(case)
    
    # Compute minimized maximum transport time
    max_time = compute_minimized_max_transport_time(A_optimized, case['X'])
    
    return {
        'Allocation Matrix': A_optimized,
        'Minimized Maximum Transport Time': max_time
    }

results = []
for i, case in enumerate(test_data):
    print(f"============= Test Case {i+1} =============")
    print(f"Description: {case['description']}")
    print(f"Priority Order: {case['priority_order']}")
    print(f"Transport Time Matrix (X): \n{case['X']}")
    print(f"Demand Array (Y): {case['Y']}")
    print(f"Stock Array (Z): {case['Z']}")
    
    # Get the results from the main function
    result = main_allocation_function(case)
    
    print("\nOptimized Allocation Matrix A:")
    print(result['Allocation Matrix'])
    
    is_valid, message = validate_allocation_priority(result['Allocation Matrix'], case['Y'], case['Z'], case['priority_order'])
    print(message)  # this will print if the allocation was valid or not

    print(f"Minimized Maximum Transport Time: {result['Minimized Maximum Transport Time']}")

    print("===========================================\n")
    
    results.append(result)



============= Test Case 1 =============
Description: All orders can be satisfied, no priority needed
Priority Order: [0 1]
Transport Time Matrix (X): 
[[4 2]
 [3 2]]
Demand Array (Y): [1 1]
Stock Array (Z): [3 4]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sudingli/miniconda3/envs/wuliu/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/edae1344ed8a4feaacd5fb11bf9297c5-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/edae1344ed8a4feaacd5fb11bf9297c5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 39 RHS
At line 48 BOUNDS
At line 53 ENDATA
Problem MODEL has 8 rows, 5 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.33333 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 inte

## Analysis

### Complexity Analysis

To analyze the complexity of your Linear Programming (LP) formulation for the allocation problem, consider the following:

1. **Variable Initialization**: The decision variable matrix \(A\) has \(m \times n\) elements. Plus, there's one extra variable \(M\). Thus, initialization is \(O(m \times n)\).

2. **Objective Function**: You're only setting \(M\) as the objective, which is \(O(1)\).

3. **Constraints**: 
   - The constraint \(M \geq X[i][j] \times A_{ij}\) runs in \(O(m \times n)\).
   - The constraint for the sum of allocations for each order is \(O(m \times n)\) because there are \(n\) orders and each sum goes through \(m\) elements.
   - The constraint for the sum of allocations from each warehouse is also \(O(m \times n)\) because there are \(m\) warehouses and each sum goes through \(n\) elements.
   
   Overall, the constraints add up to \(O(m \times n) + O(m \times n) + O(m \times n) = O(m \times n)\).

4. **Solving the LP**: The complexity of solving an integer linear programming (ILP) problem is, in general, NP-hard. However, given that you are using the PuLP solver, it's likely using simplex or a similar efficient algorithm for linear relaxations and then branching methods for the integer constraints. It's tough to pin down an exact complexity for this part due to a variety of factors (e.g., the specific solver's implementation, the problem structure, etc.). However, for many practical instances, commercial solvers can find optimal solutions quickly.

Putting it together, setting up the LP has a time complexity of \(O(m \times n)\). The solver's time complexity can vary, but the setup itself is polynomial in the size of the input.


This provides a comprehensive view of the time complexity of your LP approach.



## Next Steps:

```markdown
### Step 8: Test Current Algorithm
1. Test with various data, particularly edge cases.
2. Cases where all orders can't be satisfied.
3. Cases where there's exact stock to meet demands.
4. Situations with highly varied transportation times.

### Step 9: Alternative Algorithms
1. **Bipartite Graph Matching**: This can be seen as a variant of the assignment problem, where you're trying to assign warehouses to orders such that the cost (or time, in this context) is minimized.
    - Implement using the Hungarian method or augmenting path algorithms.
2. **Network Flow**: Model it as a min-cost max-flow problem. Each order and warehouse can be seen as nodes, with capacities and costs.
3. **Non-Heuristic Optimization**: Explore solutions using optimization libraries like `scipy.optimize` or tools like Google's OR-tools.

### Step 10: LP Data Generation for Testing
1. Create a function to generate random scenarios that adhere to the problem's constraints.
2. Make sure to include variations like:
    - Varying the number of orders and warehouses.
    - Introducing outliers in transportation times to test robustness.
    - Having excess demand or stock.

### Step 11: Comparative Analysis
1. Compare the results of the various algorithms on the same test dataset.
2. Analyze:
    - Average minimized maximum transport time.
    - Computation time.
    - Scalability with increasing orders or warehouses.
3. This will give insights into the strengths and weaknesses of each approach, helping you choose the most suitable one.

### Step 12: Documentation
1. Document the assumptions made for each approach.
2. Discuss the pros and cons of each method and provide guidelines on when to use which.

### Step 13: Optimization
1. Based on test results, try to further optimize the best-performing algorithms.
2. Explore parallel processing or other optimization techniques to reduce computation time.

### Step 14: Final Testing & Validation
1. Validate the final chosen solution on a broader test dataset.
2. Ensure that all constraints are always met.

### Step 15: Deployment
1. If you're planning to integrate this solution into a larger system or tool, work on deployment.
2. Ensure integration with other parts of your system, like databases or frontend interfaces, if applicable.

Remember, while testing different algorithms, always use the same set of test data. This ensures a fair comparison between them.

```

## improvement
### Step 8: Test Current Algorithm
1. Test with various data, particularly edge cases.
2. Cases where all orders can't be satisfied.
3. Cases where there's exact stock to meet demands.
4. Situations with highly varied transportation times.

In [ ]:
small_test_data = [
    {
        'description': 'Basic case',
        'priority_order': np.array([0, 1]),
        'X': np.array([
            [1, 3],
            [2, 1]
        ]),
        'Y': np.array([5, 5]),
        'Z': np.array([10, 10]),
        'expected_A': np.array([
            [5, 0],
            [0, 5]
        ]),
        'expected_max_time': 2
    },
    {
        'description': 'Unsatisfiable Orders',
        'priority_order': np.array([0, 1]),
        'X': np.array([
            [1, 3],
            [2, 1]
        ]),
        'Y': np.array([10, 10]),
        'Z': np.array([5, 5]),
        'expected_A': np.array([
            [5, 5],
            [0, 0]
        ]),
        'expected_max_time': 3
    },
    {
        'description': 'Exact Stock',
        'priority_order': np.array([0, 1]),
        'X': np.array([
            [1, 3],
            [2, 1]
        ]),
        'Y': np.array([5, 5]),
        'Z': np.array([5, 5]),
        'expected_A': np.array([
            [5, 0],
            [0, 5]
        ]),
        'expected_max_time': 2
    },
    {
        'description': 'Varied Transport Times',
        'priority_order': np.array([0, 1]),
        'X': np.array([
            [1, 10],
            [10, 1]
        ]),
        'Y': np.array([5, 5]),
        'Z': np.array([10, 10]),
        'expected_A': np.array([
            [5, 0],
            [0, 5]
        ]),
        'expected_max_time': 10
    },
    {
        'description': 'Priorities with Insufficient Stock',
        'priority_order': np.array([1, 0]),
        'X': np.array([
            [1, 2],
            [2, 1]
        ]),
        'Y': np.array([5, 5]),
        'Z': np.array([5, 3]),
        'expected_A': np.array([
            [3, 0],
            [2, 3]
        ]),
        'expected_max_time': 3
    }
]



In [ ]:
def test_allocation(test_data):
    results = []
    
    for i, case in enumerate(test_data):
        print(f"============= Test Case {i+1} =============")
        print(f"Description: {case['description']}")
        if 'priority_order' in case:
            print(f"Priority Order: {case['priority_order']}")
        print(f"Transport Time Matrix (X): \n{case['X']}")
        print(f"Demand Array (Y): {case['Y']}")
        print(f"Stock Array (Z): {case['Z']}")
        
        is_valid, message = validate_inputs(case['X'], case['Y'], case['Z'])
        
        if is_valid:
            # Initial Allocation
            A, remaining_Z, remaining_Y = initial_allocation(case)
            print("\nInitial Allocation Matrix A:")
            print(A)
            
            # Optimize Allocation
            A_optimized = optimize_allocation_priority(case)
            print("\nOptimized Allocation Matrix A:")
            print(A_optimized)
            
            # Validation of the Optimized Allocation
            is_valid, validation_message = validate_allocation_priority(A_optimized, case['Y'], case['Z'], case.get('priority_order', []))
            print(validation_message)
            
            # Storing Minimized Maximum Transport Time for the Test Case
            max_transport_time = np.max(A_optimized * case['X'])
            print(f"Minimized Maximum Transport Time: {max_transport_time}")
            
            # Storing results for later use or analysis
            result = {
                'Allocation Matrix': A_optimized,
                'Minimized Maximum Transport Time': max_transport_time
            }
            results.append(result)
            
        else:
            print(f"\nInvalid inputs: {message}")
        
        print("===========================================\n")
    
    return results

# You can now use:
results_small = test_allocation(small_test_data)
# results_large = test_allocation(large_test_data)




============= Test Case 1 =============
Description: Basic case
Priority Order: [0 1]
Transport Time Matrix (X): 
[[1 3]
 [2 1]]
Demand Array (Y): [5 5]
Stock Array (Z): [10 10]

Initial Allocation Matrix A:
[[5 0]
 [0 5]]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sudingli/miniconda3/envs/wuliu/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/11040df3550c442f891251413e231f12-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/11040df3550c442f891251413e231f12-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 39 RHS
At line 48 BOUNDS
At line 53 ENDATA
Problem MODEL has 8 rows, 5 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.75 - 0.00 seconds
Cgl0004I processed model has 4 rows, 3 columns (2 integer (0 of which binary)) and 8 elements
Cutoff increment increased f

In [ ]:
def generate_large_matrix(num_orders=1000, num_warehouses=1000, max_time=100):
    X = np.random.randint(1, max_time, size=(num_orders, num_warehouses))
    Y = np.random.randint(10, 100, size=num_orders)
    Z = np.random.randint(10, 100, size=num_warehouses)
    priority_order = np.arange(num_orders)
    np.random.shuffle(priority_order)
    return {
        'description': 'Large Number of Orders and Warehouses',
        'X': X,
        'Y': Y,
        'Z': Z,
        'priority_order': priority_order
    }

def generate_high_demand_and_inventory(num_orders=100, num_warehouses=100, max_demand=10000, max_inventory=10000):
    X = np.random.randint(1, 100, size=(num_orders, num_warehouses))
    Y = np.random.randint(1, max_demand, size=num_orders)
    Z = np.random.randint(1, max_inventory, size=num_warehouses)
    priority_order = np.arange(num_orders)
    np.random.shuffle(priority_order)
    return {
        'description': 'High Demand and Inventory',
        'X': X,
        'Y': Y,
        'Z': Z,
        'priority_order': priority_order
    }

def generate_combination(num_orders=500, num_warehouses=500, max_time=100, max_demand=5000, max_inventory=5000):
    X = np.random.randint(1, max_time, size=(num_orders, num_warehouses))
    Y = np.random.randint(1, max_demand, size=num_orders)
    Z = np.random.randint(1, max_inventory, size=num_warehouses)
    priority_order = np.arange(num_orders)
    np.random.shuffle(priority_order)
    return {
        'description': 'Combination of Both',
        'X': X,
        'Y': Y,
        'Z': Z,
        'priority_order': priority_order
    }

large_test_data = [
    generate_large_matrix(250,250),
    generate_high_demand_and_inventory(500,500),
    generate_combination(500,500)
]


#results_large = test_allocation(large_test_data)

============= Test Case 1 =============
Description: Large Number of Orders and Warehouses
Priority Order: [ 50 103 231  91 191 233 162 238 115 213 208   5 109  48 166 112 189 219
 121 240   6 247  55  64 120  22 143   8  49 106   9 186 243 241 145  10
 206  90 147  36 171  12  87 245 222  78  17  15  85 194 117 161  14  54
 158  83  41 110  99 226 125  58 169 135 173 215  27 133   3 211 183  65
 160 163 175 164  62   1  95  92  59 248  28 195 224 193 192 227  21  61
 202 153  20  32 198   4 200 119   0 187  88 234  76  89 123  23  51 230
  39 132  19  37 144 138  25 182 150  47 221  60 201  79  56 167  29 207
  80  97 116 203 172 142  84  44 157  69  96  26  46 105 155 114  53 108
 179 178 185 154  68  73  30  72 176 181 107  77  13 168 140 148 212 223
 218 118  33 174 229 149 184 190 165 129  75 128 249  52 237 131 134   2
  67  18  42 136  74 152 216 111 214 197 127  57 141 170  40  38 130 124
 235  66 205 236 228 196 232  45 156  24 217  63  71  34  31  70 139 244
 225 151  93 104 

PulpSolverError: Pulp: Error while trying to execute, use msg=True for more details/home/sudingli/miniconda3/envs/wuliu/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc

the above error is due to the input data is too large, the solver can't handle it, so we need to find a way to reduce the input data size, or find a better solver.